In [81]:
import lightgbm as lgb

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [82]:
df_train = pd.read_csv('train.csv')

In [83]:
df_train.head()

,TXKEY,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,T43428,41505.0,-16.526507,8.584972,-18.649853,9.505594,-13.793819,-2.832404,-16.701694,7.517344,...,1.190739,-1.127670,-2.358579,0.673461,-1.413700,-0.462762,-2.018575,-1.042804,364.19,1
1,T49906,44261.0,0.339812,-2.743745,-0.134070,-1.385729,-1.451413,1.015887,-0.524379,0.224060,...,-0.213436,-0.942525,-0.526819,-1.156992,0.311211,-0.746647,0.040996,0.102038,520.12,0
2,T29474,35484.0,1.399590,-0.590701,0.168619,-1.029950,-0.539806,0.040444,-0.712567,0.002299,...,0.102398,0.168269,-0.166639,-0.810250,0.505083,-0.232340,0.011409,0.004634,31.00,0
3,T276481,167123.0,-0.432071,1.647895,-1.669361,-0.349504,0.785785,-0.630647,0.276990,0.586025,...,0.358932,0.873663,-0.178642,-0.017171,-0.207392,-0.157756,-0.237386,0.001934,1.50,0
4,T278846,168473.0,2.014160,-0.137394,-1.015839,0.327269,-0.182179,-0.956571,0.043241,-0.160746,...,-0.238644,-0.616400,0.347045,0.061561,-0.360196,0.174730,-0.078043,-0.070571,0.89,0


In [84]:
x = df_train.drop(['TXKEY','Class'],axis=1)
y = df_train['Class']

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.33, random_state=42)

In [86]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [87]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 64,
    'num_trees': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [88]:
# number of leaves,will be used in feature transformation
num_leaf = 64

In [89]:
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_train)

print('Save model...')
# save model to file
gbm.save_model('model.txt')

Start training...


/Users/llc/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.00625827
[2]	training's binary_logloss: 0.0060161
[3]	training's binary_logloss: 0.00582705
[4]	training's binary_logloss: 0.00564377
[5]	training's binary_logloss: 0.0054866
[6]	training's binary_logloss: 0.00533254
[7]	training's binary_logloss: 0.00520926
[8]	training's binary_logloss: 0.00509753
[9]	training's binary_logloss: 0.0049962
[10]	training's binary_logloss: 0.00490579
[11]	training's binary_logloss: 0.00478799
[12]	training's binary_logloss: 0.00468698
[13]	training's binary_logloss: 0.00459341
[14]	training's binary_logloss: 0.00450223
[15]	training's binary_logloss: 0.00441051
[16]	training's binary_logloss: 0.0043233
[17]	training's binary_logloss: 0.00424554
[18]	training's binary_logloss: 0.00416223
[19]	training's binary_logloss: 0.00408707
[20]	training's binary_logloss: 0.00401627
[21]	training's binary_logloss: 0.0039477
[22]	training's binary_logloss: 0.00388377
[23]	training's binary_logloss: 0.00382269
[24]	training's binary_lo

In [90]:
print('Start predicting...')
# predict and get data on leaves, training data
y_pred = gbm.predict(X_train, pred_leaf=True)

print(np.array(y_pred).shape)
print(y_pred[:3])

Start predicting...
(133573, 100)
[[58  5 43 50 33 55 54 31 55 51 55 56 56 53 49 48 48 39 42 52 44 42 41 61
  62 58 56 59 58 54 50 46 45 50 49 47 48 51 43 35 59 54 50 63 47 60 45 59
  60 53 31 63 57 62 29 40  6  1  1 30 42 44 51 44 43  1 49 57 60  1 31 29
  30 42 26 48 29 11 57 53 33 30 30 29 30 58 62 54 44 53 49 63 45 44 56 62
  43 59 40 41]
 [58 34 56 58 47 55 54 54 55 51 55 55 56 59 49 38 38 54 56 52 44 42 41 58
  62 62 60 58 62 57 50 45 45 49 49 47 48 51 43 46 54 50 61 46 43 60 45 59
  60 53 57 60 54 53 56 58  6 54 53 30 42 44 45 44 43 62 49 57 50 18 61 58
  58 63 55 48 29 11 57 53 33 30 30 29 30 58 51 54 44 53 49 63 62 61 56 62
  43 59 40 41]
 [21 34 56 58 18 55 54 54 55 51 56 55 57 27 55 38 38 54 56  2 44 42 41 58
  23 62 61 58 62 57 59 45  5 49 19 34 37 34 35 46 54 50 61 46 43 60 45 59
  60 53 57 60 54 53 56 58  1 54 53  2 40 44 59 39 40 62 36  3 60 18 59  6
   6 13  6 17 16  1 57  1  2 54  2  2  2 58 51 54 44 53  9 37 62 61 56  1
   1  1  1  1]]


In [91]:
print('Writing transformed training data')
transformed_training_matrix = np.zeros([len(y_pred), len(y_pred[0]) * num_leaf],
                                       dtype=np.int64)  # N * num_tress * num_leafs

Writing transformed training data


In [92]:
for i in range(0, len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
    transformed_training_matrix[i][temp] += 1

In [94]:
y_pred = gbm.predict(X_val, pred_leaf=True)
print('Writing transformed validating data')
transformed_validating_matrix = np.zeros([len(y_pred), len(y_pred[0]) * num_leaf], dtype=np.int64)
for i in range(0, len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
    transformed_validating_matrix[i][temp] += 1

Writing transformed validating data


In [95]:
lr = LogisticRegression(penalty='l2',C=0.05) # logistic model construction
lr.fit(transformed_training_matrix,y_train)  # fitting the data
y_pred_val = lr.predict(transformed_validating_matrix)   # Give the probabilty on each label

In [96]:
f1_score(y_val,y_pred_val,average=None)

array([0.99979451, 0.85714286])

## 把Train和Val資料合在一起建模 

In [106]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(x,y)

In [107]:
print('Start predicting...')
# predict and get data on leaves, training data
y_pred = gbm.predict(x, pred_leaf=True)

Start predicting...


In [108]:
print('Writing transformed training data')
transformed_training_matrix = np.zeros([len(y_pred), len(y_pred[0]) * num_leaf],
                                       dtype=np.int64)  # N * num_tress * num_leafs

Writing transformed training data


In [109]:
for i in range(0, len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
    transformed_training_matrix[i][temp] += 1

In [111]:
lr = LogisticRegression(penalty='l2',C=0.05) # logistic model construction
lr.fit(transformed_training_matrix,y)  # fitting the data

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [113]:
y_pred_train = lr.predict(transformed_training_matrix)   # Give the probabilty on each label
f1_score(y,y_pred_train,average=None)

array([0.99993217, 0.95839753])

## 測試資料

In [114]:
df_test = pd.read_csv('test_public.csv')

In [115]:
TXKEY =np.array(df_test["TXKEY"])
TXKEY

array(['T2', 'T5', 'T7', ..., 'T284797', 'T284800', 'T284802'],
      dtype=object)

In [116]:
X_test = df_test.drop(['TXKEY'],axis=1)

In [117]:
y_pred = gbm.predict(X_test, pred_leaf=True)

print(np.array(y_pred).shape)
print(y_pred[:3])

(85443, 100)
[[21 34 57 59 18 44 47 54 44 60 55 55 56 27 47 38 38 55 57  2 44 42 41 51
  23 29 42 45 55 49 59 45 54 49 41 41 44 41 44 46 54 48 61 46 44 48 43 47
  58 59 57 60 54 53 56 58 37 54 53  2 40 45 59 39 40 62 50 18 25 18 59  6
   6 13  6 45 56 48 46 33 33 30 30 29 30 42 51 36 39 36 22 62 62 61  8 39
  43 44 41 41]
 [58 34 56 58 47 55 54 54 55 51 55 55 56 59 49 38 38 54 56 52 44 42 41 58
  62 62 60 58 62 57 50 45 45 49 49 47 48 51 43 46 54 50 61 46 43 60 45 59
  60 53 57 60 54 53 56 58  6 54 53 30 42 44 45 44 43 62 49 57 60 18 61 58
  58 63 55 48 29 11 57 53 33 30 30 29 30 58 51 54 44 53 49 63 62 61 56 62
  43 59 40 41]
 [42  5 43 62 38 55 36 31  6  5 39 34 33 37 49 48 48 26 30 52 60 58 57 56
  62  2 40 43  2  2 37 46 45 50 50 47 48 45 43 35 62 50  1  1 43 60 45 59
  60 53 31 63 57 62 29 40  6 17 16 30 55 55  8 34 57 13 49  9 60 11 17 15
  15 42 14 48 29 11 27 15 33 30 30 29 30 58  6 54 16 53 49 63 23 17 56 38
   8 43 40  8]]


In [118]:
transformed_testing_matrix = np.zeros([len(y_pred), len(y_pred[0]) * num_leaf], dtype=np.int64)
for i in range(0, len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
    transformed_testing_matrix[i][temp] += 1

In [119]:
y_pred_test = lr.predict(transformed_testing_matrix)   # Give the probabilty on each label

In [120]:
my_solution = pd.DataFrame(y_pred_test, TXKEY, columns = ["Class"])
my_solution

,Class
T2,0
T5,0
T7,0
T9,0
T10,0
T13,0
T15,0
T17,0
T18,0
T19,0


In [121]:
my_solution.to_csv("Ans/solution_14.csv", index_label = ["TXKEY"])